# Study 1: Six-Dimensional Affect Framework Validation

This notebook tests whether the six affect dimensions are empirically distinguishable.

## Key Predictions
1. Factor analysis should support a 6-factor model
2. Dimensions should show predicted correlation structure
3. Cross-modal convergence (self-report ~ physiology ~ neural)

## Falsification Criteria
- If a 2-factor model fits equally well
- If dimensions are not distinguishable (load on same factors)
- If cross-modal correlations are near zero

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

# Our custom modules
import sys
sys.path.insert(0, '../src')

from inevitability_empirical.affect import AffectState, PREDICTED_SIGNATURES, DIMENSION_SPECS
from inevitability_empirical.analysis import (
    test_dimension_independence,
    simulate_affect_trajectory,
    power_analysis_correlation
)

np.random.seed(42)
plt.style.use('seaborn-v0_8-whitegrid')

## 1. Power Analysis

How many participants do we need to detect the predicted correlations?

In [ ]:
# Expected effect sizes based on prior literature
expected_effects = {
    "dimension_correlation": 0.3,  # Moderate correlation between related dimensions
    "valence_hrv": 0.25,  # Valence-HRV relationship
    "sm_meditation": 0.35,  # Self-model salience reduction in meditation
}

for name, r in expected_effects.items():
    n = power_analysis_correlation(r, alpha=0.05, power=0.80)
    n_95 = power_analysis_correlation(r, alpha=0.05, power=0.95)
    print(f"{name}: r={r:.2f} → N={n} (80% power), N={n_95} (95% power)")

## 2. Simulate Data for Method Development

Before collecting real data, we simulate trajectories to test our analysis pipeline.

In [ ]:
# Simulate 100 participants with different attractors
n_participants = 100
n_observations_per_person = 50

# 25 per attractor type
attractor_types = ["neutral"] * 25 + ["depression"] * 25 + ["anxiety"] * 25 + ["flow"] * 25

all_data = []
participant_info = []

for i, attractor in enumerate(attractor_types):
    trajectory = simulate_affect_trajectory(
        n_timesteps=n_observations_per_person,
        attractor=attractor,
        noise_scale=0.15
    )
    for t, obs in enumerate(trajectory):
        all_data.append({
            "participant": i,
            "observation": t,
            "attractor": attractor,
            "valence": obs[0],
            "arousal": obs[1],
            "integration": obs[2],
            "effective_rank": obs[3],
            "counterfactual_weight": obs[4],
            "self_model_salience": obs[5],
        })

df = pd.DataFrame(all_data)
print(f"Total observations: {len(df)}")
print(f"Participants: {df['participant'].nunique()}")
df.head()

## 3. Factor Analysis: Are Six Dimensions Distinguishable?

In [ ]:
# Aggregate to person-level means for factor analysis
dimensions = ["valence", "arousal", "integration", "effective_rank", 
              "counterfactual_weight", "self_model_salience"]

person_means = df.groupby("participant")[dimensions].mean()
data_matrix = person_means.values

# Run factor analysis
fa_result = test_dimension_independence(data_matrix, dimensions)

print("Fit Indices:")
for name, value in fa_result.fit_indices.items():
    print(f"  {name}: {value:.3f}")

print(f"\nVariance explained by first 6 factors: {fa_result.variance_explained[:6].sum():.1%}")

In [ ]:
# Plot variance explained
fig, ax = plt.subplots(figsize=(8, 5))
ax.bar(range(1, 7), fa_result.variance_explained[:6] * 100)
ax.axhline(100/6, color='red', linestyle='--', label='Equal contribution')
ax.set_xlabel('Factor')
ax.set_ylabel('Variance Explained (%)')
ax.set_title('Variance Explained by Each Factor')
ax.legend()
plt.tight_layout()
plt.show()

## 4. Correlation Structure

Do the dimensions correlate as predicted by theory?

In [ ]:
# Compute correlation matrix
corr_matrix = person_means.corr()

# Plot
fig, ax = plt.subplots(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, cmap='RdBu_r', center=0, 
            vmin=-1, vmax=1, ax=ax, fmt='.2f')
ax.set_title('Correlation Matrix: Six Affect Dimensions')
plt.tight_layout()
plt.show()

In [ ]:
# Test specific predictions
predictions = [
    ("Valence-Arousal should be weak", "valence", "arousal", 0.1, 0.3),
    ("Integration-EffRank should be positive", "integration", "effective_rank", 0.2, 0.5),
    ("CF-SM should be positive", "counterfactual_weight", "self_model_salience", 0.2, 0.5),
    ("Valence-SM should be negative", "valence", "self_model_salience", -0.5, -0.1),
]

print("Testing Theoretical Predictions:")
print("=" * 60)
for desc, dim1, dim2, expected_low, expected_high in predictions:
    r = corr_matrix.loc[dim1, dim2]
    in_range = expected_low <= r <= expected_high
    status = "✓ SUPPORTED" if in_range else "✗ VIOLATED"
    print(f"{desc}:")
    print(f"  Expected: [{expected_low:.2f}, {expected_high:.2f}], Observed: {r:.2f} → {status}")
    print()

## 5. Affect Signatures by Condition

Do different psychological states produce distinct affect signatures?

In [ ]:
# Compute mean affect profile by attractor type
attractor_profiles = df.groupby("attractor")[dimensions].mean()

# Create radar/spider plot
from math import pi

def radar_plot(data, labels, title):
    N = len(labels)
    angles = [n / float(N) * 2 * pi for n in range(N)]
    angles += angles[:1]
    
    fig, ax = plt.subplots(figsize=(8, 8), subplot_kw=dict(polar=True))
    
    colors = plt.cm.Set2.colors
    for i, (name, values) in enumerate(data.items()):
        values = list(values) + [values[0]]
        ax.plot(angles, values, 'o-', linewidth=2, label=name, color=colors[i])
        ax.fill(angles, values, alpha=0.1, color=colors[i])
    
    ax.set_xticks(angles[:-1])
    ax.set_xticklabels(labels)
    ax.set_title(title)
    ax.legend(loc='upper right', bbox_to_anchor=(1.3, 1.0))
    return fig

# Prepare data
radar_data = {}
for attractor in attractor_profiles.index:
    # Normalize to [0, 1] for visualization
    vals = attractor_profiles.loc[attractor].values
    vals_normalized = (vals + 1) / 2  # Map [-1,1] to [0,1]
    radar_data[attractor.capitalize()] = vals_normalized

short_labels = ["Val", "Ar", "Φ", "r_eff", "CF", "SM"]
fig = radar_plot(radar_data, short_labels, "Affect Signatures by Psychological State")
plt.tight_layout()
plt.show()

## 6. Statistical Tests: Group Differences

In [ ]:
# ANOVA for each dimension across attractor types
print("One-way ANOVA: Dimension ~ Attractor Type")
print("=" * 60)

for dim in dimensions:
    groups = [df[df['attractor'] == att][dim].values for att in df['attractor'].unique()]
    f_stat, p_val = stats.f_oneway(*groups)
    eta_sq = f_stat / (f_stat + len(df) - len(groups))  # Approximation
    
    print(f"{dim}:")
    print(f"  F = {f_stat:.2f}, p = {p_val:.2e}, η² ≈ {eta_sq:.3f}")
    print()

## 7. Conclusions & Next Steps

### Results Summary
- Factor analysis supports [N]-factor model
- [X/Y] theoretical predictions were supported
- Groups show distinct affect signatures as predicted

### Falsification Status
- [ ] 6-factor model fit acceptable (RMSEA < 0.06, CFI > 0.95)
- [ ] Dimension correlations match theory
- [ ] Group signatures distinguishable

### Next Steps
1. Collect real experience sampling data
2. Add physiological measures for cross-modal validation
3. Extend to clinical populations